# 3 — Preprocess + helpers (ColumnTransformer, pipeline, CV)

Notebook này định nghĩa:
- `preprocess` bằng **ColumnTransformer** (impute + scale + one-hot)
- danh sách mô hình baseline (`get_base_models()`)
- hàm build pipeline + hàm đánh giá CV (`cv_evaluate`)

**Phụ thuộc:** đã chạy `app/01_load_clean_split.ipynb` (để có `X_train`, `y_train`).


## Preprocess (numeric + categorical)

In [ ]:
# 8) Xây dựng preprocess bằng ColumnTransformer: xử lý numeric/categorical (impute + scale/ohe) để đưa vào pipeline model

def make_ohe():
    # Tạo OneHotEncoder tương thích nhiều version sklearn:
    # - sklearn mới dùng sparse_output
    # - sklearn cũ dùng sparse
    try:
        return OneHotEncoder(handle_unknown="ignore", sparse_output=False)
    except TypeError:
        return OneHotEncoder(handle_unknown="ignore", sparse=False)

# Chọn các cột categorical nếu chúng tồn tại trong X (tránh lỗi nếu thiếu cột)
categorical_cols = [
    c for c in ["gender","education","currentSmoker","BPMeds","prevalentStroke","prevalentHyp","diabetes"]
    if c in X.columns
]

# Chọn các cột numeric nếu chúng tồn tại trong X
numeric_cols = [
    c for c in ["age","cigsPerDay","totChol","sysBP","diaBP","BMI","heartRate","glucose"]
    if c in X.columns
]

# Pipeline cho numeric:
# - Điền missing bằng median (ít nhạy với outlier)
# - Scale về cùng thang đo (quan trọng với LR/SVM/KNN...)
numeric_tf = SkPipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

# Pipeline cho categorical:
# - Điền missing bằng mode (giá trị xuất hiện nhiều nhất)
# - One-hot encode, ignore category mới khi gặp ở test
categorical_tf = SkPipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", make_ohe())
])

# ColumnTransformer: áp pipeline numeric cho numeric_cols, categorical cho categorical_cols
preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_tf, numeric_cols),
        ("cat", categorical_tf, categorical_cols),
    ]
)

# In ra để kiểm tra danh sách cột đang được xử lý
print("Numeric:", numeric_cols)
print("Categorical:", categorical_cols)


## Danh sách mô hình baseline

In [ ]:
# Helper: tạo dict các baseline models (init sẵn hyperparams cơ bản + xử lý imbalanced bằng class_weight khi phù hợp)

def get_base_models():
    models = {}

    # Logistic Regression: tăng max_iter để chắc chắn hội tụ; class_weight balanced để cân bằng lớp
    models["LR"]  = LogisticRegression(max_iter=2000, class_weight="balanced", random_state=SEED)

    # GaussianNB: baseline đơn giản, không cần nhiều hyperparams
    models["GNB"] = GaussianNB()

    # KNN: để mặc định (sẽ tuning sau nếu cần)
    models["KNN"] = KNeighborsClassifier()

    # SVC (RBF): class_weight balanced cho imbalanced; (chú ý: muốn predict_proba cần probability=True)
    models["SVC"] = SVC(C=1.0, gamma="scale", kernel="rbf", class_weight="balanced")

    # Decision Tree: class_weight balanced để giảm bias về majority class
    models["DT"]  = DecisionTreeClassifier(class_weight="balanced", random_state=SEED)

    # Random Forest: nhiều cây hơn để ổn định; balanced_subsample cân bằng theo từng bootstrap sample
    models["RF"]  = RandomForestClassifier(
        n_estimators=300,
        class_weight="balanced_subsample",
        random_state=SEED,
        n_jobs=-1
    )

    # Extra Trees: tương tự RF nhưng random mạnh hơn; thường baseline tốt, chạy nhanh
    models["ET"]  = ExtraTreesClassifier(
        n_estimators=500,
        class_weight="balanced_subsample",
        random_state=SEED,
        n_jobs=-1
    )

    # AdaBoost: baseline boosting (không có class_weight trực tiếp như LR/Tree)
    models["ADA"] = AdaBoostClassifier(random_state=SEED)

    # Gradient Boosting (classic)
    models["GB"]  = GradientBoostingClassifier(random_state=SEED)

    # HistGradientBoosting: boosting dạng histogram, thường nhanh hơn trên data lớn
    models["HGB"] = HistGradientBoostingClassifier(random_state=SEED)

    # Chỉ thêm XGBoost nếu đã cài (tránh lỗi import/runtime)
    if HAS_XGB:
        models["XGB"] = XGBClassifier(
            n_estimators=500,
            learning_rate=0.05,
            max_depth=4,
            subsample=0.8,
            colsample_bytree=0.8,
            eval_metric="logloss",
            random_state=SEED,
            n_jobs=-1
        )

    # Chỉ thêm LightGBM nếu đã cài
    if HAS_LGBM:
        models["LGBM"] = LGBMClassifier(
            n_estimators=800,
            learning_rate=0.03,
            num_leaves=31,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=SEED,
            n_jobs=-1
        )

    # Trả về dict để loop chạy CV baseline/tuning dễ dàng
    return models


## Pipeline + Cross-Validation evaluation

In [ ]:
# Tạo pipeline (preprocess + optional sampler + model) và hàm chạy CV để đánh giá nhiều metrics

# Chọn Pipeline class:
# - Nếu có imbalanced-learn: dùng ImbPipeline để nhúng sampler vào pipeline đúng chuẩn (tránh data leakage)
# - Nếu không: dùng sklearn Pipeline
PIPE_CLS = ImbPipeline if HAS_IMBLEARN else SkPipeline

def build_pipeline(clf, use_sampler=True):
    # Bước 1: luôn luôn preprocess (impute/scale/ohe) trước khi train model
    steps = [("preprocess", preprocess)]

    # Bước 2 (tuỳ chọn): oversample để cân bằng class (chỉ hợp lệ khi có imblearn)
    if HAS_IMBLEARN and use_sampler:
        steps.append(("sampler", RandomOverSampler(random_state=SEED)))
    elif use_sampler and (not HAS_IMBLEARN):
        # Nếu thiếu imblearn thì không resampling được trong pipeline -> cảnh báo để bạn biết
        print("WARNING: imbalanced-learn missing -> no resampling; using class_weight where possible.")

    # Bước 3: gắn classifier cuối pipeline
    steps.append(("model", clf))

    # Trả về pipeline hoàn chỉnh để fit/predict/CV
    return PIPE_CLS(steps)

# Bộ metric dùng trong cross_validate (đánh giá nhiều tiêu chí cùng lúc)
scoring = {
    "pr_auc": "average_precision",      # PR-AUC (tốt cho imbalanced)
    "roc_auc": "roc_auc",               # ROC-AUC
    "recall": "recall",                 # Recall
    "precision": "precision",           # Precision
    "f1": "f1",                         # F1-score
    "bal_acc": "balanced_accuracy"      # Balanced Accuracy
}

def cv_evaluate(model_name, pipe, Xtr, ytr, cv):
    # Dict kết quả tối thiểu luôn có tên model để dễ tổng hợp bảng
    out = {"model": model_name}
    try:
        # Chạy cross-validation:
        # - fit pipeline trên train folds
        # - evaluate trên valid fold
        # - trả về scores cho từng metric ở từng fold
        res = cross_validate(
            pipe, Xtr, ytr,
            cv=cv,
            scoring=scoring,
            n_jobs=-1,
            error_score="raise"   # nếu lỗi thì raise để catch ở except và log error rõ ràng
        )

        # Lấy trung bình score qua các folds cho mỗi metric
        for k in scoring.keys():
            out[k] = float(np.mean(res[f"test_{k}"]))

        # Trả về kết quả + None (không lỗi)
        return out, None

    except Exception as e:
        # Nếu lỗi (vd model không tương thích/scoring fail), trả về dict + message lỗi để debug
        return out, str(e)
